IDENYA ADALAH FILE KELUAR MASUK
1. proses dulu si cosine doang outputnya list down mana x_test yg udah ada di y_train (score apa nya tinggi gitu) dan buat file baru dengan label yes/no
2. proses ini ngambil data dari proses no 1 khusus yang no aja jadi yg udah bener ga usah diproses lagi...
3. terus begitu sampe terakhir model ML jalan

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from rapidfuzz import process, fuzz

# Load data
train_df = pd.read_csv("data/Hospital_Train.csv")
test_df = pd.read_csv("data/Hospital_Test.csv")

# Clean data
train_df_clean = train_df.dropna(subset=["Hospital Name rev 2"])
X_train = train_df_clean["Hospital_Name (clean)"]
y_train = train_df_clean["Hospital Name rev 2"]
X_test = test_df["Hospital_Name (clean)"]
y_test = test_df["Hospital Name rev 2"]

# TF-IDF vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Cosine similarity prediction
cos_sim_matrix = cosine_similarity(X_test_tfidf, X_train_tfidf)
cosine_top_indices = cos_sim_matrix.argmax(axis=1)
cosine_preds = y_train.iloc[cosine_top_indices].values
cosine_top_scores = cos_sim_matrix.max(axis=1)

# Format clean name
def format_clean_name(name):
    return re.sub(r'[^\w\s&]', '', name).upper().strip()

clean_names = X_test.apply(format_clean_name)

# FINAL PREDICTION LOGIC
final_preds = []
fuzzy_preds = []
fuzzy_scores = []
tfidf_zero_flags = []
tfidf_term_details = []

feature_names = vectorizer.get_feature_names_out()

for i in range(len(X_test)):
    cleaned_input = clean_names.iloc[i]
    actual = y_test.iloc[i]
    tfidf_row = X_test_tfidf[i]

    # Token analyzer
    tokens = vectorizer.build_analyzer()(cleaned_input)

    # TF-IDF vector content
    indices = tfidf_row.nonzero()[1]
    values = tfidf_row.data
    tfidf_tokens = {feature_names[idx] for idx in indices}
    tfidf_score_map = {feature_names[idx]: round(val, 4) for idx, val in zip(indices, values)}
    tfidf_term_details.append(tfidf_score_map)

    # Cek apakah ada token hilang
    missing_tokens = set(tokens) - tfidf_tokens
    has_zero = len(missing_tokens) > 0
    tfidf_zero_flags.append(has_zero)

    # Fuzzy matching
    fuzzy_match, fuzzy_score, _ = process.extractOne(
        cleaned_input,
        y_train.tolist(),  # fix: convert to list
        scorer=fuzz.token_sort_ratio
    )
    fuzzy_preds.append(fuzzy_match)
    fuzzy_scores.append(fuzzy_score)

    # FINAL DECISION FLOW
    if cosine_preds[i] == actual:
        final_preds.append(cosine_preds[i])
    elif has_zero:
        # Use fuzzy result as new input to cosine
        new_input_vector = vectorizer.transform([fuzzy_match])
        new_cos_sim = cosine_similarity(new_input_vector, X_train_tfidf)
        new_top_idx = new_cos_sim.argmax()
        new_cosine_pred = y_train.iloc[new_top_idx]

        if new_cosine_pred == actual:
            final_preds.append(new_cosine_pred)
            # print(f"[{i}] 🌀 Fuzzy → Cosine match: {fuzzy_match} → {new_cosine_pred}")
        else:
            final_preds.append(cleaned_input)
    elif fuzzy_match == actual:
        final_preds.append(fuzzy_match)
    else:
        final_preds.append(cleaned_input)

# Buat hasil dataframe
result_df = pd.DataFrame({
    "Hospital_Name_clean": X_test,
    "Actual": y_test,
    "Cosine_Pred": cosine_preds,
    "Fuzzy_Pred": fuzzy_preds,
    "Fuzzy_Score": fuzzy_scores,
    "Clean_Pred": clean_names,
    "Final_Pred": final_preds,
    "Cosine_Score": cosine_top_scores,
    "TF-IDF_Zero_Found": tfidf_zero_flags,
    "TF-IDF_Terms_Score": tfidf_term_details
})

# Bersihkan spasi ganda
for col in ["Final_Pred", "Fuzzy_Pred"]:
    result_df[col] = result_df[col].str.replace(r"\s+", " ", regex=True).str.strip()

# Tampilkan mismatch & evaluasi
mismatch_df = result_df[result_df["Actual"] != result_df["Final_Pred"]]
accuracy = accuracy_score(y_test, result_df["Final_Pred"])

print("\n🎯 Accuracy:", accuracy)
print("🔥 Total mismatch:", len(mismatch_df))
print("🧠 Total TF-IDF OOV:", sum(tfidf_zero_flags))
mismatch_df.head(10)



🎯 Accuracy: 0.8883495145631068
🔥 Total mismatch: 23
🧠 Total TF-IDF OOV: 148


,Hospital_Name_clean,Actual,Cosine_Pred,Fuzzy_Pred,Fuzzy_Score,Clean_Pred,Final_Pred,Cosine_Score,TF-IDF_Zero_Found,TF-IDF_Terms_Score
16,RS KHUSUS ORTHOEDI KARIMA UTAMA,RS KHUSUS ORTHOPEDI KARIMA UTAMA,RS KHUSUS BEDAH KARIMA UTAMA,RS KHUSUS ORTHOPEDI KARIMA UTAMA,98.412698,RS KHUSUS ORTHOEDI KARIMA UTAMA,RS KHUSUS ORTHOEDI KARIMA UTAMA,0.771471,True,"{'karima': 0.543, 'karima utama': 0.543, 'khus..."
34,RS THT PROKLAMASI,RS KHUSUS THT BEDAH PROKLAMASI,RS THT BEDAH PROKLAMASI,RS PROKLAMASI,86.666667,RS THT PROKLAMASI,RS THT PROKLAMASI,0.614295,False,"{'proklamasi': 0.4631, 'rs': 0.0869, 'rs tht':..."
40,HERMINA HOSPITALS MADIUN,RS HERMINA MADIUN,RS SANTA CLARA MADIUN,HARI NATIONAL HOSPITAL,69.565217,HERMINA HOSPITALS MADIUN,HERMINA HOSPITALS MADIUN,0.398326,True,"{'hermina': 0.4645, 'hospitals': 0.4444, 'madi..."
66,RSIA BUN,RS BUNDA JAKARTA,RS BUN,RS BUN,85.714286,RSIA BUN,RSIA BUN,0.587421,True,"{'bun': 0.8989, 'rsia': 0.4382}"
67,REVIEW DOR 22 -DOKUMEN LAPORAN OP SUDAH DIKIRI...,RS BETHESDA YOGYAKARTA,RS BETHESDA,RSUP PROF DR R D KANDOU MANADO,48.000000,REVIEW DOR 22 DOKUMEN LAPORAN OP SUDAH DIKIRIM...,REVIEW DOR 22 DOKUMEN LAPORAN OP SUDAH DIKIRIM...,0.388503,True,"{'22': 0.3309, 'bethesda': 0.2697, 'by': 0.344..."
88,RSSTELISABETH SEMARANG,RS ST ELISABETH SEMARANG,SILOAM HOSPITAL SEMARANG,RS ELISABETH SEMARANG,83.720930,RSSTELISABETH SEMARANG,RSSTELISABETH SEMARANG,0.533078,True,{'semarang': 1.0}
108,"THE ENT,VOICE DAN SNORING CLINIC",MOUNT ELIZABETH MEDICAL CENTRE SINGAPORE,THE ENT SPECIALIST CENTRE PARKWAY SINGAPORE,HE ENDOCRINE CLINIC,68.000000,THE ENTVOICE DAN SNORING CLINIC,THE ENTVOICE DAN SNORING CLINIC,0.890817,True,"{'clinic': 0.1893, 'dan': 0.2692, 'ent': 0.294..."
110,THE SPORE GASTROENTEROLOGY & LIVER INTERNIST C...,SPORE GASTROENTEROLOGY & LIVER INTERNIST CENTR...,SINGAPORE GASTROENTEROLOGY & LIVER INTERNIST C...,SINGAPORE GASTROENTEROLOGY & LIVER INTERNIST C...,78.571429,THE SPORE GASTROENTEROLOGY & LIVER INTERNIST C...,THE SPORE GASTROENTEROLOGY & LIVER INTERNIST C...,0.868745,True,"{'care': 0.2188, 'dan': 0.2315, 'gastroenterol..."
111,SPORTSLN ORHOPEDIC CLINIC GELEAGLES MEDICAL CA...,GLENEAGLES HOSPITAL,E-CLINIC,SOG ANDY TAN CLINIC FOR WOMEN GLENEAGLES MEDIC...,64.761905,SPORTSLN ORHOPEDIC CLINIC GELEAGLES MEDICAL CA...,SPORTSLN ORHOPEDIC CLINIC GELEAGLES MEDICAL CA...,0.364512,True,"{'cantre': 0.8493, 'clinic': 0.3645, 'medical'..."
112,KLINIK AP DAN AP,LABORATORIUM DAN KLINIK PRAMITA,RS PREMIER BINTARO,KLINIK ANANDA,75.862069,KLINIK AP DAN AP,KLINIK AP DAN AP,0.422959,True,"{'ap': 0.9199, 'dan': 0.3316, 'klinik': 0.2093}"


In [2]:
# Optional: save to Excel
result_df.to_excel("NLP_Cosine_Fuzzy_Pred.xlsx", index=False)
print("✅ Predictions saved to 'NLP_Cosine_Fuzzy_Pred.xlsx'")

✅ Predictions saved to 'NLP_Cosine_Fuzzy_Pred.xlsx'


In [3]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score
import pandas as pd

# Load data hasil sebelumnya
result_df = pd.read_excel("NLP_Cosine_Fuzzy_Pred.xlsx")
result_df["Final_Pred"] = result_df["Final_Pred"].astype(str).str.strip()
result_df["Actual"] = result_df["Actual"].astype(str).str.strip()

# Step 1: Ambil data yg sudah cocok
correct_df = result_df[result_df["Actual"] == result_df["Final_Pred"]].copy()
X_train_sbert = correct_df["Hospital_Name_clean"].tolist()
y_train_sbert = correct_df["Actual"].tolist()

# Step 2: Ambil data mismatch
mismatch_df = result_df[result_df["Actual"] != result_df["Final_Pred"]].copy()
X_test_sbert = mismatch_df["Hospital_Name_clean"].tolist()
y_test_sbert = mismatch_df["Actual"].tolist()

# Step 3: Embedding SBERT
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
X_train_emb = sbert_model.encode(X_train_sbert, convert_to_tensor=True)
X_test_emb = sbert_model.encode(X_test_sbert, convert_to_tensor=True)

# Step 4: Cosine similarity pakai SBERT
sbert_preds = []
for test_vec in X_test_emb:
    cos_scores = util.cos_sim(test_vec, X_train_emb)[0]
    top_idx = cos_scores.argmax().item()
    sbert_preds.append(y_train_sbert[top_idx])

# Step 5: Update prediksi dan simpan SBERT_Pred
for i, idx in enumerate(mismatch_df.index):
    result_df.loc[idx, "SBERT_Pred"] = sbert_preds[i]
    result_df.loc[idx, "Final_Pred"] = sbert_preds[i]

# Step 6: Bersihkan spasi
result_df["Final_Pred"] = result_df["Final_Pred"].str.replace(r"\s+", " ", regex=True).str.strip()
result_df["SBERT_Pred"] = result_df["SBERT_Pred"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()

# Step 7: Evaluasi ulang
mismatch_df_final = result_df[result_df["Actual"] != result_df["Final_Pred"]][
    ["Hospital_Name_clean", "Actual", "Cosine_Pred", "Fuzzy_Pred", "SBERT_Pred", "Final_Pred"]
]
accuracy_final = accuracy_score(result_df["Actual"], result_df["Final_Pred"])

# Output
print("✅ AFTER SBERT:")
print("🎯 Accuracy:", accuracy_final)
print("🔥 Remaining mismatch:", len(mismatch_df_final))
mismatch_df_final.head(10)


✅ AFTER SBERT:
🎯 Accuracy: 0.8980582524271845
🔥 Remaining mismatch: 21


,Hospital_Name_clean,Actual,Cosine_Pred,Fuzzy_Pred,SBERT_Pred,Final_Pred
16,RS KHUSUS ORTHOEDI KARIMA UTAMA,RS KHUSUS ORTHOPEDI KARIMA UTAMA,RS KHUSUS BEDAH KARIMA UTAMA,RS KHUSUS ORTHOPEDI KARIMA UTAMA,RS KHUSUS ORTHOPEDI RSOP CIAMIS,RS KHUSUS ORTHOPEDI RSOP CIAMIS
34,RS THT PROKLAMASI,RS KHUSUS THT BEDAH PROKLAMASI,RS THT BEDAH PROKLAMASI,RS PROKLAMASI,RS KHUSUS ORTHOPEDI RSOP CIAMIS,RS KHUSUS ORTHOPEDI RSOP CIAMIS
40,HERMINA HOSPITALS MADIUN,RS HERMINA MADIUN,RS SANTA CLARA MADIUN,HARI NATIONAL HOSPITAL,BRAWIJAYA HOSPITAL SAHARJO,BRAWIJAYA HOSPITAL SAHARJO
66,RSIA BUN,RS BUNDA JAKARTA,RS BUN,RS BUN,RSUD DR ISKAK TULUNGAGUNG,RSUD DR ISKAK TULUNGAGUNG
67,REVIEW DOR 22 -DOKUMEN LAPORAN OP SUDAH DIKIRI...,RS BETHESDA YOGYAKARTA,RS BETHESDA,RSUP PROF DR R D KANDOU MANADO,RS GRHA KEDOYA,RS GRHA KEDOYA
110,THE SPORE GASTROENTEROLOGY & LIVER INTERNIST C...,SPORE GASTROENTEROLOGY & LIVER INTERNIST CENTR...,SINGAPORE GASTROENTEROLOGY & LIVER INTERNIST C...,SINGAPORE GASTROENTEROLOGY & LIVER INTERNIST C...,PAEDIATRIC & GASTROENTEROLOGY CLINIC,PAEDIATRIC & GASTROENTEROLOGY CLINIC
111,SPORTSLN ORHOPEDIC CLINIC GELEAGLES MEDICAL CA...,GLENEAGLES HOSPITAL,E-CLINIC,SOG ANDY TAN CLINIC FOR WOMEN GLENEAGLES MEDIC...,CLINIC UNIVERSITY OF NOTTINGHAM,CLINIC UNIVERSITY OF NOTTINGHAM
112,KLINIK AP DAN AP,LABORATORIUM DAN KLINIK PRAMITA,RS PREMIER BINTARO,KLINIK ANANDA,APOTEK & LAB KLINIK DEWANI,APOTEK & LAB KLINIK DEWANI
113,SUNWAY MEDICAL CENTRE KUALA LUMPUR,SUNWAY MEDICAL CENTRE SDN BHD MALAYSIA,I HEAL MEDICAL CENTRE MALAYSIA,PARKCITY MEDICAL CENTRE KUALA LUMPUR,GLENEGLES HOSPITAL KUALA LUMPUR,GLENEGLES HOSPITAL KUALA LUMPUR
116,EYE ANDEAR HOSPITAL,EYE AND EAR HOSPITAL,RS EYE,FUDA CANCER HOSPITAL,ADVANCED EYE CLINIC AND SURGERY,ADVANCED EYE CLINIC AND SURGERY


In [4]:
# Optional: save to Excel
result_df.to_excel("NLP_Cosine_Fuzzy_SBERT_Pred.xlsx", index=False)
print("✅ Predictions saved to 'NLP_Cosine_Fuzzy_SBERT_Pred.xlsx'")

✅ Predictions saved to 'NLP_Cosine_Fuzzy_SBERT_Pred.xlsx'
